# 🎬 Make Content (Phymac) - Procesar Contenido

Este notebook automatiza la transformación de debates técnicos y pedagógicos en contenido de micro-aprendizaje. 

**Pasos:**
1. Configurar el entorno (Instalar dependencias).
2. Conectar Google Drive (para videos de entrada y resultados).
3. Procesar: Limpieza de audio + Transcripción + Recorte.
4. Generar metadatos para la edición visual.

## 1. Instalación y Configuración

In [ ]:
# Instalar dependencias necesarias
!pip install -q faster-whisper deepfilternet click torch torchaudio

In [ ]:
import os
import sys
from pathlib import Path

# Configurar rutas
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_ROOT = Path('/content/drive/MyDrive/contenido-automatico')
    sys.path.append(str(PROJECT_ROOT))
else:
    # En local, asumimos que estamos en notebooks/ y el root es el padre
    PROJECT_ROOT = Path(os.getcwd()).parent
    if str(PROJECT_ROOT) not in sys.path:
        sys.path.append(str(PROJECT_ROOT))

print(f"Ruta del proyecto: {PROJECT_ROOT}")

## 2. Verificar Hardware (GPU)

In [ ]:
from src.utils import check_env
info = check_env()

## 3. Configuración del Proceso

Ajusta los parámetros para el video que quieres procesar.

In [ ]:
# @title Configuración de Ingesta { display-mode: "form" }

INPUT_VIDEO = "samples/sesion_1.mp4" # @param {type:"string"}
START_TIME = "00:00:00" # @param {type:"string"}
END_TIME = "00:00:30" # @param {type:"string"}
NARRATIVE = "Fail & Fix" # @param ["Fail & Fix", "Discovery & Learn", "Debate & Decision", "Challenge & Build", "Iteration & Refinement"]
SKIP_CLEAN = False # @param {type:"boolean"}
OUTPUT_DIR = "output" # @param {type:"string"}

input_path = PROJECT_ROOT / INPUT_VIDEO
output_path = PROJECT_ROOT / OUTPUT_DIR

print(f"Video de entrada: {input_path}")
print(f"Carpeta de salida: {output_path}")

## 4. Ejecutar Pipeline

In [ ]:
from src.ingest import ingest
from click.testing import CliRunner

# Usamos CliRunner para ejecutar la función de click cómodamente
runner = CliRunner()
args = [
    str(input_path),
    '--start', START_TIME,
    '--end', END_TIME,
    '--output_dir', str(output_path),
    '--narrative', NARRATIVE
]

if SKIP_CLEAN:
    args.append('--skip-clean')

print("[*] Iniciando proceso...")
result = runner.invoke(ingest, args)
print(result.output)

## 5. Ver Resultados

In [ ]:
import json
from IPython.display import Video, Audio, display

video_name = Path(INPUT_VIDEO).stem
meta_path = output_path / f"{video_name}_meta.json"

if meta_path.exists():
    with open(meta_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"### Resumen: {data['narrativa']} ###")
    print(f"Idioma detectado: {data['language']}")
    
    # Mostrar primeras palabras de la transcripción
    print("\nTranscripción (primeras 20 palabras):")
    text = " ".join([w['word'] for w in data['palabras'][:20]])
    print(f"{text}...")
else:
    print("[!] No se encontró el archivo de metadatos. Revisa los errores arriba.")